In [1]:
%pylab inline
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data.dataloader as dataloader
import torch.optim as optim

from torch.utils.data import TensorDataset
from torch.autograd import Variable
from torchvision import transforms
from torchvision.datasets import MNIST

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

Populating the interactive namespace from numpy and matplotlib


ModuleNotFoundError: No module named 'torch'

## Data

* [Pytorch Transform Documentation](http://pytorch.org/docs/torchvision/transforms.html)


1. **torchvision.transforms.Compose:** Composes several transforms together. 
2. **torchvision.transforms.ToTensor:** Converts a PIL Image or numpy.ndarray (H x W x C) in the range [0, 255] to a torch.FloatTensor of shape (C x H x W) in the range [0.0, 1.0]. 
3. **dataloader.DataLoader:** [https://pytorch.org/docs/stable/data.html] Combines a dataset and a sampler, and provides single- or multi-process iterators over the dataset.
4. **torch.nn.functionnal :** [http://pytorch.org/docs/_modules/torch/nn/functional.html]

In [ ]:
# Téléchargement des données
train = MNIST('./data', train=True, download=True, transform=transforms.Compose([
    transforms.ToTensor(), # ToTensor effectue une normalisation min-max.
]), )

test = MNIST('./data', train=False, download=True, transform=transforms.Compose([
    transforms.ToTensor(), # ToTensor effectue une normalisation min-max.
]), )

# Création du DataLoader
dataloader_args = dict(shuffle=True, batch_size=64, num_workers=1, pin_memory=True)
train_loader = dataloader.DataLoader(train, **dataloader_args)
test_loader = dataloader.DataLoader(test, **dataloader_args)

In [ ]:
# Affichage des informations des corpus

train_data = train.train_data
train_data = train.transform(train_data.numpy())
test_data = test.test_data
test_data = test.transform(test_data.numpy())

print('[Train]')
print(' - Numpy Shape:', train.train_data.cpu().numpy().shape)
print(' - Tensor Shape:', train.train_data.size())
print(' - Transformed Shape:', train_data.size())
print(' - min:', torch.min(train_data))
print(' - max:', torch.max(train_data))
print(' - mean:', torch.mean(train_data))
print(' - std:', torch.std(train_data))
print(' - var:', torch.var(train_data))
print('[Test]')
print(' - Numpy Shape:', test.test_data.cpu().numpy().shape)
print(' - Tensor Shape:', test.test_data.size())
print(' - Transformed Shape:', test_data.size())
print(' - min:', torch.min(test_data))
print(' - max:', torch.max(test_data))
print(' - mean:', torch.mean(test_data))
print(' - std:', torch.std(test_data))
print(' - var:', torch.var(test_data))

## Affichage de quelques images

In [ ]:
fig=plt.figure(figsize=(20, 8))
columns = 8
rows = 1
for i in range(1, columns*rows +1):
    idx = np.random.randint(60000)
    fig.add_subplot(rows, columns, i)
    plt.imshow(train.train_data[idx])
plt.show()

## Modèle feed-forward (pour exemple)

In [ ]:
# Model FeedForward pour exemple
class FFModel(nn.Module):
    def __init__(self):
        super(FFModel, self).__init__()
        
        self.fc1 = nn.Linear(784, 548)
        self.bc1 = nn.BatchNorm1d(548)
        self.fc2 = nn.Linear(548, 252)
        self.bc2 = nn.BatchNorm1d(252)
        self.fc3 = nn.Linear(252, 10)
        
    def forward(self, x):
        # flatten 28x28 = 784
        x = x.view((-1, 784))
        # couche entièrement connectée 784 -> 548
        h = self.fc1(x)
        # batch normalization
        h = self.bc1(h)
        # ReLU
        h = F.relu(h)
        # Dropout avec probabilité .5
        h = F.dropout(h, p=0.5, training=self.training)
        
        # couche entièrement connectée 548 -> 252
        h = self.fc2(h)
        h = self.bc2(h)
        h = F.relu(h)
        h = F.dropout(h, p=0.2, training=self.training)
        
        # couche entièrement connectée 252 -> 10
        h = self.fc3(h)
        # estimation de la distribution de probabilités avec softmax
        out = F.log_softmax(h, dim=1)
        return out

ff_model = FFModel()
#model.cuda() # CUDA! (si on a un GPU)
ff_optimizer = optim.Adam(ff_model.parameters(), lr=0.001)

## Modèle convolutionnel (à compléter)

In [ ]:
# Convolutionnal model
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        
        # TODO: déclaration des variables membres utilisées dans forward()
        
    def forward(self, x):
        
        # TODO: Exemple de modèle
        # Q0: Afficher la taille des données x, vous devez trouver 64x1x28x28
        # 1.1 Convolution 2d avec 5 filtres (channels). Noyau de taille 5
        # Q1.1: Quelle sera la taille après convolution ?
        # 1.2 Max pooling (taille du noyau = 2) + ReLU
        # Q1.2: Quelle sera la taille cette opération ?
        
        # 2.1 Convolution 2d avec 10 filtres (channels). Noyau de taille 5
        # Q2.1: Quelle sera la taille après convolution ?
        # 2.2 Dropout
        # -> voir torch.nn.Dropout2d
        # 2.3 Max pooling (taille du noyau = 2) + ReLU
        # -> voir torch.nn.Functionnal.max_pool2d, torch.nn.Functionnal.ReLU
        # Q1.2: Quelle sera la taille cette opération ?
        
        # 3. Couche entièrement connectée (essayer linéaire, non-linéaire), taille de sortie au choix
        # 4.1 Couche entièrement connectée (essayer linéaire, non-linéaire), taille de sortie = 10
        # -> voir torch.nn.Linear, torch.nn.Linear, torch.nn.Tanh, torch.nn.Sigmoid
        # 4.2 return softmax -> torch.nn.Functionnal.log_softmax
        
        return result

    
cnn_model = CNNModel()
#cnn_model.cuda() # CUDA! @
cnn_optimizer = optim.Adam(cnn_model.parameters(), lr=0.001)

## Train - Boucle principale d'entrainement

In [ ]:
num_epochs=3

cnn_model.train()
cnn_losses = []

for epoch in range(num_epochs):
    for batch_idx, (data, target) in enumerate(train_loader):
        # Get Samples
        #data, target = Variable(data.cuda()), Variable(target.cuda())
        data, target = Variable(data), Variable(target)
        
        # TODO: Remise à zéro des gradients
        
        # TODO: Prédiction
        pred = cnn_model(data) 
        # Calculer la cross_entropy loss
        # -> voir torch.nn.Functionnal.cross_entropy
        loss = ...
        
        # Sauvegarde des losses pour affichage
        cnn_losses.append(loss.data.item())
        
        # Backpropagation
        loss.backward()
        optimizer.step()
        
        # Affichage
        if batch_idx % 100 == 0 or batch_idx % 100 == 1 :
            print('\r Train Epoch: {} [{}/{} ({:.0f}%)]\t Loss: {:.6f}'.format(
                    epoch, 
                    batch_idx * len(data), 
                    len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), 
                    loss.data.item()), 
                    end='')
            
    print()

 ## Affichage de l'erreur

In [ ]:
from scipy.signal import savgol_filter
import ipywidgets as widgets
from ipywidgets import interact

x = np.linspace(0, len(cnn_losses), len(cnn_losses))
fig = plt.figure(figsize=(13, 8)) 
ax = fig.add_subplot(1,1,1)
cnn_line, = ax.plot(x, cnn_losses)

def update_losses(smooth=51):
    cnn_line.set_ydata(savgol_filter(cnn_losses, smooth, 3))
    fig.canvas.draw()

interact(update_losses, smooth=(5, 201, 2));


## Evaluate

In [ ]:
# CUDA (si on a des GPUs)
#evaluate_x = Variable(test_loader.dataset.test_data.type_as(torch.FloatTensor())).cuda()
#evaluate_y = Variable(test_loader.dataset.test_labels).cuda()
evaluate_x = Variable(test_loader.dataset.test_data.type_as(torch.FloatTensor()))
evaluate_x = evaluate_x.reshape(10000, 1, 28, 28)
evaluate_y = Variable(test_loader.dataset.test_labels)

# TODO: appliquer le modèle sur les données de test
output = ...
# TODO récupérer les classes les plus probables
pred = ...
# calculer la précision
d = pred.eq(evaluate_y.data).cpu()
#print("d:", d, " sum:", d.sum())
accuracy = d.sum().type_as(torch.FloatTensor())/d.size()[0]

print('{} Accuracy:'.format(model_type), accuracy)